In [57]:
# All the imports
from dotenv import load_dotenv
from langchain.tools import Tool, tool
from langchain.chat_models import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from huggingface_hub import login
from transformers import pipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor, LLMSingleActionAgent, create_structured_chat_agent,ZeroShotAgent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_retrieval_chain, LLMChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
import os

In [58]:
# Initializing the LLM and hugging face
load_dotenv()
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
login(os.getenv('HUGGINGFACE_API_KEY'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\VisahanSritharan\.cache\huggingface\token
Login successful


In [59]:
# Outlining all the tools
@tool
def search_tool(query : str):
    """Performs online searches to answer the user query."""
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def summarizaion_tool(query : str):
    """This tool takes a long text and summarizes it"""
    summarizer = pipeline("summarization", model="Falconsai/text_summarization")
    summary = summarizer(query, max_length=1000, min_length=30, do_sample=False)
    return summary[0]['summary_text']

tools = [
    Tool(name='search_tool', func=search_tool, description="""Performs online searches to answer the user query."""),
    Tool(name = 'summarization_tool', func=summarizaion_tool, description="""This tool takes a long text and summarizes it""")
]

In [69]:
prefix = """Have a conversation with a human, answer the questions as best as you can. You have access to the following tools """
suffix = """
Begin!
{chat_history}
Question: {input}
{agent_scratchpad}
"""


#query = 'What was Apples gross margin in 2022'
#query = 'What is the weather in London today'


#query = 'Give me all the stock allocations for client 3'
prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    input_variables=['input', 'agent_scratchpad', 'tools', 'tool_names', 'history'], 
    prefix=prefix,
    suffix=suffix
   )

llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)

llm_chain = LLMChain(llm=llm, prompt=prompt)
memory = ConversationBufferWindowMemory(memory_key='chat_history', k=5)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools = tools,
    verbose = True,
)



agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose = True,
    memory = memory
)



In [75]:
agent_executor.invoke('What is cheaper, a flight to Lisbon or London')
                      
                      



> Entering new AgentExecutor chain...
Thought: I need to compare the flight ticket prices to Lisbon and London.
Action: search_tool
Action Input: "flight ticket prices from New York to Lisbon"
Observation: Thu, 30 Jan SWF - LIS with Fly Play. 1 stop. Mon, 3 Feb LIS - SWF with Fly Play. 1 stop. from £293. Lisbon. £300 per passenger.Departing Tue, 11 Feb, returning Sun, 16 Feb.Return flight with TAP Air Portugal.Outbound indirect flight with TAP Air Portugal, departs from New York Newark on Tue, 11 Feb, arriving in Lisbon.Inbound indirect flight ... The cheapest return flight ticket from New York John F Kennedy Airport to Lisbon found by KAYAK users in the last 72 hours was for $751 on TAP AIR PORTUGAL, followed by Azores Airlines ($753). ... KAYAK's flight Price Forecast tool uses historical data to determine whether the price for a flight to Lisbon from New York John F Kennedy Airport ... Compare flight deals to Lisbon from New York LaGuardia from over 1,000 providers. Then choose th

{'input': 'What is cheaper, a flight to Lisbon or London',
 'chat_history': 'Human: What is the weather in London\nAI: The current weather in London is sunny with a high of 17°C and a low of 10°C. There is a gentle breeze expected throughout the day. UV index is 4.\nHuman: Is this warmer than Barcelona\nAI: The current weather in London is sunny with a high of 17°C and a low of 10°C, while the current temperature in Barcelona is not specified in the search results.\nHuman: What about in comparison to Lisbon\nAI: The current weather in London is sunny with a high of 17°C and a low of 10°C, while Lisbon has a forecast of temperatures as high as 24°C with a 40% chance of precipitation.\nHuman: How much are flight tickets there from New York\nAI: Flight ticket prices from New York to London can range from $441 to $710 round-trip on average.\nHuman: IS this cheaper than a flight to London\nAI: The best price for a return flight from New York to Barcelona is £291, while flight ticket prices 